In [1]:
import os
import glob
from sklearn.model_selection import train_test_split

In [2]:
from scipy.spatial.qhull import QhullError
from scipy import spatial
spatial.QhullError = QhullError

/tmp/ipykernel_42/2927579722.py:1: DeprecationWarning: Please use `QhullError` from the `scipy.spatial` namespace, the `scipy.spatial.qhull` namespace is deprecated.
  from scipy.spatial.qhull import QhullError


In [3]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
from category_encoders import OrdinalEncoder

In [4]:
import os
import glob

def get_image_paths(folder_path):
    image_paths = []
    for filename in os.listdir(folder_path):
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(folder_path, filename)
            image_paths.append(image_path)
    return image_paths

if __name__ == "__main__":
    # Specify input folders
    input_folder_blacksigatoka = r"/kaggle/input/blddisease-wise/BLD_augmented_data/BlackSigatoka"
    input_folder_furasiumwilt = r"/kaggle/input/blddisease-wise/BLD_augmented_data/FurasiumWilt"
    input_folder_cordana = r"/kaggle/input/blddisease-wise/BLD_augmented_data/Cordana"

    # Get image paths for each class
    image_paths_blacksigatoka = get_image_paths(input_folder_blacksigatoka)
    image_paths_furasiumwilt = get_image_paths(input_folder_furasiumwilt)
    image_paths_cordana = get_image_paths(input_folder_cordana)

    # Print the first few image paths for each class
    print("Image paths for BlackSigatoka:")
    print(image_paths_blacksigatoka[:5])

    print("\nImage paths for FurasiumWilt:")
    print(image_paths_furasiumwilt[:5])

    print("\nImage paths for Cordana:")
    print(image_paths_cordana[:5])


Image paths for BlackSigatoka:
['/kaggle/input/blddisease-wise/BLD_augmented_data/BlackSigatoka/257_jpeg.rf.3056e638b36d2805190c731d35590a75_aug_0.png', '/kaggle/input/blddisease-wise/BLD_augmented_data/BlackSigatoka/267_jpeg.rf.b449a72add174c60aaef9595acd9e1f0_aug_0.png', '/kaggle/input/blddisease-wise/BLD_augmented_data/BlackSigatoka/Image_1650_aug_0.png', '/kaggle/input/blddisease-wise/BLD_augmented_data/BlackSigatoka/Image_1672_aug_1.png', '/kaggle/input/blddisease-wise/BLD_augmented_data/BlackSigatoka/202_jpeg.rf.4cae05ab9f7463e25bc30a6a4fdd0883_aug_0.png']

Image paths for FurasiumWilt:
['/kaggle/input/blddisease-wise/BLD_augmented_data/FurasiumWilt/Image_3243_aug_0.png', '/kaggle/input/blddisease-wise/BLD_augmented_data/FurasiumWilt/Image_106_aug_1.png', '/kaggle/input/blddisease-wise/BLD_augmented_data/FurasiumWilt/Image_2922_aug_0.png', '/kaggle/input/blddisease-wise/BLD_augmented_data/FurasiumWilt/Image_3277_aug_1.png', '/kaggle/input/blddisease-wise/BLD_augmented_data/Furasi

In [6]:
import os
import cv2
import numpy as np
from skimage import exposure
from tqdm import tqdm

def preprocess_images(file_paths, output_folder, target_size=(64, 64)):
    # Ensure output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for path in tqdm(file_paths):
        # Read the image
        image = cv2.imread(path)

        # Convert to grayscale
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Apply Histogram Equalization to enhance contrast
        equalized = exposure.equalize_hist(gray)

        # Resize the image to a standard size
        resized = cv2.resize(equalized, target_size)

        # Normalize pixel values to be between 0 and 1
        resized = resized / 255.0

        # Save the preprocessed image
        filename = os.path.basename(path)
        output_path = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}_preprocessed.png")
        cv2.imwrite(output_path, (resized * 255).astype(np.uint8))

if __name__ == "__main__":
    # Specify input and output folders
    input_folder_blacksigatoka = r"/kaggle/input/blddisease-wise/BLD_augmented_data/BlackSigatoka"
    input_folder_furasiumwilt = r"/kaggle/input/blddisease-wise/BLD_augmented_data/FurasiumWilt"
    input_folder_cordana = r"/kaggle/input/blddisease-wise/BLD_augmented_data/Cordana"
    output_folder_cordana = r"/kaggle/working/BLD_augmented_data2/Cordana"
    output_folder_blacksigatoka = r"/kaggle/working/BLD_augmented_data2/BlackSigatoka"
    output_folder_furasiumwilt = r"/kaggle/working/BLD_augmented_data2/FurasiumWilt"

    # Get image paths for each class
    image_paths_blacksigatoka = [os.path.join(input_folder_blacksigatoka, filename) for filename in os.listdir(input_folder_blacksigatoka)]
    image_paths_furasiumwilt = [os.path.join(input_folder_furasiumwilt, filename) for filename in os.listdir(input_folder_furasiumwilt)]
    image_paths_cordana = [os.path.join(input_folder_cordana, filename) for filename in os.listdir(input_folder_cordana)]

    # Preprocess images
    preprocess_images(image_paths_blacksigatoka, output_folder_blacksigatoka)
    preprocess_images(image_paths_furasiumwilt, output_folder_furasiumwilt)
    preprocess_images(image_paths_cordana, output_folder_cordana)


100%|██████████| 324/324 [00:04<00:00, 64.95it/s]


In [7]:
# import os
# import cv2
# import imgaug.augmenters as iaa
# from tqdm import tqdm

# def augment_images(input_folder, output_folder, num_copies=4):
#     # Ensure output folder exists
#     if not os.path.exists(output_folder):
#         os.makedirs(output_folder)

#     # Define augmentation pipeline
#     seq = iaa.Sequential([
#         iaa.Affine(rotate=(-90, 90)),        # 90-degree rotation
#         iaa.Fliplr(1.0),                     # Flip left-right
#         iaa.ShearX(shear=(-15, 15)),           # Shear rotation up to 10%
#         iaa.Crop(percent=(0, 0.1)),           # Crop up to 10%
#         iaa.Affine(rotate=(-180, 180)),
#        #iaa.Affine(rotate=(-360, 360)),
#         #iaa.Affine(scale=(0.8, 1.5)),
#         #iaa.GaussianBlur(sigma=(0.0, 1.0)),
#         #iaa.Multiply((0.8, 1.5), per_channel=0.2),
#         #iaa.MultiplyHue((0.9, 1.6)),
#     ])

#     # Loop through each image in the input folder
#     for filename in tqdm(os.listdir(input_folder)):
#         if filename.endswith(('.png', '.jpg', '.jpeg')):
#             image_path = os.path.join(input_folder, filename)

#             # Read the image
#             image = cv2.imread(image_path)

#             # Apply augmentation and save augmented copies
#             for i in range(num_copies):
#                 augmented_image = seq.augment_image(image)
#                 output_path = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}_aug_{i}.png")
#                 cv2.imwrite(output_path, augmented_image)

# if __name__ == "__main__":
#     # Specify input and output folders
#     input_folder_blacksigatoka = r"/kaggle/input/blddisease-wise/BLD_augmented_data/BlackSigatoka"
#     input_folder_furasiumwilt = r"/kaggle/input/blddisease-wise/BLD_augmented_data/FurasiumWilt"
#     input_folder_cordana=r"/kaggle/input/blddisease-wise/BLD_augmented_data/Cordana"
#     output_folder_cordana=r"/kaggle/working/BLD_augmented_data2/Cordana"
#     output_folder_blacksigatoka = r"/kaggle/working/BLD_augmented_data2/BlackSigatoka"
#     output_folder_furasiumwilt = r"/kaggle/working/BLD_augmented_data2/FurasiumWilt"

#     # Specify the number of copies per image
#     #num_copies = 2

#     # Perform augmentations
#     augment_images(input_folder_blacksigatoka, output_folder_blacksigatoka, 2)
#     augment_images(input_folder_furasiumwilt, output_folder_furasiumwilt, 2)
#     augment_images(input_folder_cordana, output_folder_cordana, 8)


In [8]:
# import os
# import cv2
# import numpy as np
# from sklearn.model_selection import train_test_split
# # from sklearn.svm import SVC
# from sklearn.metrics import accuracy_score
# from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
# print("Succesfully imported packeges")

In [9]:
# def extract_hog_lbp_features(image_path):
#     image = io.imread(image_path, as_gray=True)
#     image = (image * 255).astype(np.uint8)
#     hog_features, _ = feature.hog(image, visualize=True, block_norm='L2-Hys')
#     lbp_features = feature.local_binary_pattern(image, P=8, R=1, method='uniform')
#     return np.concatenate((hog_features, lbp_features.flatten()))

# batch_size = 50

# # Extract HOG and LBP features for the training set
# X_train_features = []

# for i in range(0, len(X_train), batch_size):
#     batch_images = X_train[i:i+batch_size]
#     batch_features = [extract_hog_lbp_features(image_path) for image_path in batch_images]
#     X_train_features.extend(batch_features)

# # Extract HOG and LBP features for the testing set
# X_test_features = []

# for i in range(0, len(X_test), batch_size):
#     batch_images = X_test[i:i+batch_size]
#     batch_features = [extract_hog_lbp_features(image_path) for image_path in batch_images]
#     X_test_features.extend(batch_features)

# # Convert to NumPy arrays
# hog_xfeatures_train = np.array(X_train_features)
# lbp_xfeatures_train = np.array(X_test_features)

# # Flatten the features
# hog_features_flat_train = hog_xfeatures_train.flatten()
# lbp_features_flat_train = lbp_xfeatures_train.flatten()

# # Convert to NumPy arrays
# hog_xfeatures_test = np.array(X_test_features)
# lbp_xfeatures_test = np.array(X_test_features)

# # Flatten the features
# hog_features_flat_test = hog_xfeatures_test.flatten()
# lbp_features_flat_test = lbp_xfeatures_test.flatten()

# # Concatenate flattened features into a single vector
# combined_features_train = np.concatenate((hog_features_flat_train, lbp_features_flat_train))

# combined_features_test  = np.concatenate((hog_features_flat_test,lbp_features_flat_test))


# # Print the shape of the combined features
# print("Shape of combined features (training set):", combined_features_train.shape)

# print("Shape of combined features (training set):", combined_features_test.shape)

# # Print the first few values of the combined features
# print("First few values of combined features (training set):", combined_features_train[:])

# print("Shape of combined features (training set):", combined_features_test[:])


In [10]:
import os
import glob
from skimage import io, feature
import numpy as np

def extract_hog_lbp_features(image_path):
    image = io.imread(image_path, as_gray=True)
    image = (image * 255).astype(np.uint8)
    hog_features, _ = feature.hog(image, visualize=True, block_norm='L2-Hys')
    lbp_features = feature.local_binary_pattern(image, P=8, R=1, method='uniform')
    return np.concatenate((hog_features, lbp_features.flatten()))

def extract_features(input_folder):
    file_paths = glob.glob(os.path.join(input_folder, '*.png'))

    features = []
    for image_path in file_paths:
        feature_vector = extract_hog_lbp_features(image_path)
        features.append(feature_vector)

    return np.array(features)

if __name__ == "__main__":
    # Specify input and output folders
    input_folder_blacksigatoka = r"/kaggle/working/BLD_augmented_data2/BlackSigatoka"
    input_folder_furasiumwilt = r"/kaggle/working/BLD_augmented_data2/FurasiumWilt"
    input_folder_cordana = r"/kaggle/working/BLD_augmented_data2/Cordana"

    # Extract HOG and LBP features
    hog_lbp_features_blacksigatoka = extract_features(input_folder_blacksigatoka)
    hog_lbp_features_furasiumwilt = extract_features(input_folder_furasiumwilt)
    hog_lbp_features_cordana = extract_features(input_folder_cordana)

    # Print the shapes of the extracted features
    print("Shape of HOG and LBP features (BlackSigatoka):", hog_lbp_features_blacksigatoka.shape)
    print("Shape of HOG and LBP features (FurasiumWilt):", hog_lbp_features_furasiumwilt.shape)
    print("Shape of HOG and LBP features (Cordana):", hog_lbp_features_cordana.shape)


Shape of HOG and LBP features (BlackSigatoka): (1008, 7012)
Shape of HOG and LBP features (FurasiumWilt): (1082, 7012)
Shape of HOG and LBP features (Cordana): (324, 7012)


In [13]:
import os
import glob
from skimage import io, feature
import numpy as np

X_train = []  # Initialize X_train
y_train = []  # Initialize y_train

def extract_hog_lbp_features(image_path):
    image = io.imread(image_path, as_gray=True)
    image = (image * 255).astype(np.uint8)
    hog_features, _ = feature.hog(image, visualize=True, block_norm='L2-Hys')
    lbp_features = feature.local_binary_pattern(image, P=8, R=1, method='uniform')
    return np.concatenate((hog_features, lbp_features.flatten()))

def extract_features(input_folder):
    file_paths = glob.glob(os.path.join(input_folder, '*.png'))
    for image_path in file_paths:
        features = extract_hog_lbp_features(image_path)
        X_train.append(features)
        # Assuming the label is the name of the class folder
        label = os.path.basename(os.path.dirname(image_path))
        y_train.append(label)

if __name__ == "__main__":
    # Specify input and output folders
    input_folder_blacksigatoka = r"/kaggle/working/BLD_augmented_data2/BlackSigatoka"
    input_folder_furasiumwilt = r"/kaggle/working/BLD_augmented_data2/FurasiumWilt"
    input_folder_cordana = r"/kaggle/working/BLD_augmented_data2/Cordana"

    # Extract HOG and LBP features
    extract_features(input_folder_blacksigatoka)
    extract_features(input_folder_furasiumwilt)
    extract_features(input_folder_cordana)

    # Convert lists to arrays
    X_train_array = np.array(X_train)
    y_train_array = np.array(y_train)

    # Concatenate features and labels
    all_features = np.column_stack((X_train_array, y_train_array))

    # Flatten the features
    flattened_features = all_features.flatten()

    # Print the shapes of the extracted features
    print("Shape of HOG and LBP features (All Classes):", X_train_array.shape)
    print("Shape of labels (All Classes):", y_train_array.shape)
    print("Shape of concatenated features and labels:", all_features.shape)
    print("Shape of flattened features (All Classes):", flattened_features.shape)


Shape of HOG and LBP features (All Classes): (2414, 7012)
Shape of labels (All Classes): (2414,)
Shape of concatenated features and labels: (2414, 7013)
Shape of flattened features (All Classes): (16929382,)


In [14]:
import numpy as np
from skimage import feature, io
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from tensorflow import keras
from tensorflow.keras import layers

In [15]:
# def load_data(folder_paths, label_mapping):
#     X = []
#     y = []

#     for label, folder_key in label_mapping.items():
#         class_folder = os.path.join(folder_paths[folder_key], label)
#         features = extract_features(class_folder)  # Modify this to your extraction function
#         labels = [label] * len(features)

#         X.extend(features)
#         y.extend(labels)

#     return np.array(X), np.array(y)

# if __name__ == "__main__":
#     # Specify input folders
#     folder_paths = {
#         'BlackSigatoka': 'BlackSigatoka',
#         'FurasiumWilt': 'FurasiumWilt',
#         'Cordana': 'Cordana'
#     }

#     # Specify label mapping
#     label_mapping = {'BlackSigatoka': 'BlackSigatoka', 'FurasiumWilt': 'FurasiumWilt', 'Cordana': 'Cordana'}

#     # Load data
#     X, y = load_data(folder_paths, label_mapping)

#     # Print the shape of loaded data
#     print("Shape of features:", X.shape)
#     print("Shape of labels:", y.shape)


In [19]:
# Apply z-score normalization
scaler = StandardScaler()
X_train_normalized = scaler.fit_transform(X_train_array)
all_features = np.column_stack((X_train_normalized, y_train_array))

# Print the shape and first few values of the normalized features
print("Shape of normalized features (training set):", X_train_normalized.shape)
print("Shape of all features:", all_features.shape)
print("First few values of all features:", all_features[:])
print("First few values of normalized features (training set):", X_train_normalized[:])

Shape of normalized features (training set): (2414, 7012)
Shape of all features: (2414, 7013)
First few values of all features: [['-0.16912662598197656' '0.0' '-0.11398855726176177' ...
  '0.041981492915668925' '0.04324788304874349' 'BlackSigatoka']
 ['-0.16912662598197656' '0.0' '-0.11398855726176177' ...
  '0.041981492915668925' '0.04324788304874349' 'BlackSigatoka']
 ['-0.16912662598197656' '0.0' '-0.11398855726176177' ...
  '0.041981492915668925' '0.04324788304874349' 'BlackSigatoka']
 ...
 ['-0.16912662598197656' '0.0' '-0.11398855726176177' ...
  '0.041981492915668925' '0.04324788304874349' 'Cordana']
 ['-0.16912662598197656' '0.0' '-0.11398855726176177' ...
  '0.041981492915668925' '0.04324788304874349' 'Cordana']
 ['-0.16912662598197656' '0.0' '-0.11398855726176177' ...
  '0.041981492915668925' '0.04324788304874349' 'Cordana']]
First few values of normalized features (training set): [[-0.16912663  0.         -0.11398856 ...  0.04834056  0.04198149
   0.04324788]
 [-0.16912663  

In [ ]:
import glob
from sklearn.model_selection import train_test_split

# Now, let's split the data
X_train_split, X_test_split, y_train_split, y_test_split = train_test_split(X_train_normalized, y_train, test_size=0.2, random_state=42)

# Print the sizes of the training and testing sets
print(f"Training set size: {len(X_train)}")
print(f"Testing set size: {len(X_test)}")


In [ ]:
# def extract_hog_lbp_features(image_path):
#     image = io.imread(image_path, as_gray=True)
#     image = (image * 255).astype(np.uint8)
#     hog_features, _ = feature.hog(image, visualize=True, block_norm='L2-Hys')
#     lbp_features = feature.local_binary_pattern(image, P=8, R=1, method='uniform')
#     return np.concatenate((hog_features, lbp_features.flatten()))

# batch_size = 50

# # Extract HOG and LBP features for the training set
# X_train_features = []

# for i in range(0, len(X_train), batch_size):
#     batch_images = X_train[i:i+batch_size]
#     batch_features = [extract_hog_lbp_features(image_path) for image_path in batch_images]
#     X_train_features.extend(batch_features)

# # Extract HOG and LBP features for the testing set
# X_test_features = []

# for i in range(0, len(X_test), batch_size):
#     batch_images = X_test[i:i+batch_size]
#     batch_features = [extract_hog_lbp_features(image_path) for image_path in batch_images]
#     X_test_features.extend(batch_features)

# # Convert to NumPy arrays
# hog_xfeatures_train = np.array(X_train_features)
# lbp_xfeatures_train = np.array(X_test_features)

# # Flatten the features
# hog_features_flat_train = hog_xfeatures_train.flatten()
# lbp_features_flat_train = lbp_xfeatures_train.flatten()

# # Convert to NumPy arrays
# hog_xfeatures_test = np.array(X_test_features)
# lbp_xfeatures_test = np.array(X_test_features)

# # Flatten the features
# hog_features_flat_test = hog_xfeatures_test.flatten()
# lbp_features_flat_test = lbp_xfeatures_test.flatten()

# # Concatenate flattened features into a single vector
# combined_features_train = np.concatenate((hog_features_flat_train, lbp_features_flat_train))

# combined_features_test  = np.concatenate((hog_features_flat_test,lbp_features_flat_test))


# # Print the shape of the combined features
# print("Shape of combined features (training set):", combined_features_train.shape)

# print("Shape of combined features (training set):", combined_features_test.shape)

# # Print the first few values of the combined features
# print("First few values of combined features (training set):", combined_features_train[:])

# print("Shape of combined features (training set):", combined_features_test[:])


In [ ]:
print(type(X_train))

In [ ]:
import pandas as pd
column1=['categorical_feature1']
df1 = pd.DataFrame(combined_features_train,columns=column1)

In [ ]:
print(df1.columns)
print(df1.head())

In [ ]:
import pandas as pd
#df1['categorical_feature1'] = df1['categorical_feature1'].astype('category')

In [ ]:
import pandas as pd

# Assuming df is your DataFrame
#df1['categorical_feature1'] = pd.Categorical(df1['categorical_feature1'])
#df1['categorical_feature1'] = df1['categorical_feature1'].cat.codes

In [ ]:
import pandas as pd
column2=['categorical_feature2']
df2 = pd.DataFrame(combined_features_test,columns=column2)

In [ ]:
print(df2.columns)
print(df2.head())

In [ ]:
column_types=df2.dtypes
print(column_types)

In [ ]:
import pandas as pd

# Assuming df is your DataFrame
#df2['categorical_feature2'] = df2['categorical_feature2'].astype('category')

In [ ]:
import pandas as pd

# Assuming df is your DataFrame
#df2['categorical_feature2'] = pd.Categorical(df2['categorical_feature2'])
#df2['categorical_feature2'] = df2['categorical_feature2'].cat.codes

In [ ]:
print(df2.dtypes)
print(df2.head())

In [ ]:

# Plot histograms for HOG and LBP features
import matplotlib.pyplot as plt
plt.figure(figsize=(12, 6))

plt.subplot(2, 2, 1)
plt.hist(hog_features_flat_train, bins=50, color='blue', alpha=0.7)
plt.title('HOG Features Histogram (Training Set)')

plt.subplot(2, 2, 2)
plt.hist(lbp_features_flat_train, bins=50, color='green', alpha=0.7)
plt.title('LBP Features Histogram (Training Set)')

plt.subplot(2, 2, 3)
plt.hist(hog_features_flat_test, bins=50, color='blue', alpha=0.7)
plt.title('HOG Features Histogram (Testing Set)')

plt.subplot(2, 2, 4)
plt.hist(lbp_features_flat_test, bins=50, color='green', alpha=0.7)
plt.title('LBP Features Histogram (Testing Set)')

plt.tight_layout()
plt.show()

In [ ]:
numpy_array=df1.values

In [ ]:
print("NumPy Array:")
print(numpy_array)

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(numpy_array)

In [ ]:
len(X_train_scaled)

In [ ]:
# Convert y_train, y_val, and y to NumPy arrays
y_train = np.array(y_train)
y_test = np.array(y_test)

In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from itertools import product

In [ ]:
model = Sequential()
model.add(Dense(128, input_dim=12800, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
print("X_train_scaled shape:", X_train_scaled.shape)
print("y_train shape:", np.array(y_train).shape)

In [ ]:
model.fit(X_train_scaled,y_train,epochs=50, batch_size=50)